In [1]:
from pandas import DataFrame
from py2neo import Graph,Node,Relationship,NodeMatcher
import pandas as pd
import numpy as np
import os
# 连接Neo4j数据库
uri = 'bolt://localhost:7687'
auth = ('neo4j','123')
graph = Graph(profile='http://localhost:7474/', auth = auth)

In [2]:
holder = pd.read_csv('C:/Users/Administrator/Desktop/data/enterprise_holders.csv')
stock = pd.read_csv('C:/Users/Administrator/Desktop/data/enterprise_basic.csv')
concept = pd.read_csv('C:/Users/Administrator/Desktop/data/enterprise_concept.csv')

notice = pd.read_csv('C:/Users/Administrator/Desktop/data/enterprise_notices.csv')
price = pd.read_csv('C:/Users/Administrator/Desktop/data/stock_prices.csv')

In [3]:
holder.head()

,股票代码,公告日期,报告期,股东名称,持有数量（股）,持有比例
0,600383.SH,20220430.0,20220331.0,富德生命人寿保险股份有限公司-万能H,979419165.0,21.69
1,600383.SH,20220430.0,20220331.0,深圳市福田投资控股有限公司,351777629.0,7.79
2,600383.SH,20220430.0,20220331.0,大家人寿保险股份有限公司-万能产品,245151325.0,5.43
3,600383.SH,20220430.0,20220331.0,富德生命人寿保险股份有限公司-万能G,197681270.0,4.38
4,600383.SH,20220430.0,20220331.0,富德生命人寿保险股份有限公司-分红,169889373.0,3.76


In [4]:
stock.head()

,股票代码,企业名称,行业,交易所
0,600000.SH,浦发银行,银行,上交所
1,600009.SH,上海机场,机场,上交所
2,600010.SH,包钢股份,普钢,上交所
3,600011.SH,华能国际,火力发电,上交所
4,600015.SH,华夏银行,银行,上交所


In [5]:
concept.head()

,概念名称,股票代码
0,黑洞概念,601989.SH
1,黄金水道,600018.SH
2,黄金概念,600089.SH
3,黄金概念,600362.SH
4,黄金概念,600547.SH


In [6]:
notice.head()

,股票代码,公告日期,公告标题,公告内容
0,600000.SH,20221230,上海浦东发展银行股份有限公司第七届董事会第四十六次会议决议公告,NaN
1,600000.SH,20221230,上海浦东发展银行股份有限公司独立董事关于关联交易事项的独立意见,NaN
2,600000.SH,20221230,上海浦东发展银行股份有限公司关于与上海久事（集团）有限公司关联交易公告,NaN
3,600000.SH,20221230,上海浦东发展银行股份有限公司第七届监事会第四十五次会议决议公告,NaN
4,600000.SH,20221229,上海浦东发展银行股份有限公司关于赎回次级债券的公告,NaN


In [7]:
price.head()

,股票代码,交易日期,开盘价,最高价,最低价,收盘价,成交量
0,600000.SH,20220104,8.54,8.58,8.52,8.57,377076.47
1,600000.SH,20220105,8.57,8.68,8.56,8.64,554591.35
2,600000.SH,20220106,8.66,8.66,8.56,8.57,410310.62
3,600000.SH,20220107,8.57,8.73,8.57,8.71,630182.97
4,600000.SH,20220110,8.72,8.78,8.66,8.72,340553.12


In [8]:
# 股东实体
for i in holder.values:
   a = Node('股东',TS代码=i[0],股东名称=i[3],持股数量=i[4],持股比例=i[5])
   #print('TS代码:'+str(i[0]),'股东名称:'+str(i[3]),'持股数量:'+str(i[4]))
   graph.create(a)

# 股票实体
for i in stock.values:
   a = Node('股票',TS代码=i[0],企业名称=i[1],行业=i[2], 交易所=i[3])
   # print('TS代码:'+str(i[0]),'股票名称:'+str(i[1]),'行业:'+str(i[2]))
   graph.create(a)

# 概念实体
for i in concept.values:
   a = Node('概念',概念名称=i[0],股票代码=i[1])
   # print('概念:'+str(i[0]),'股票代码:'+str(i[1]))
   graph.create(a)

# 行业实体
for i in stock.values:
   a = Node('行业',行业=i[2],企业名称=i[1],TS代码=i[0])
   graph.create(a)
    
# 价格实体
for i in price.values:
   a = Node('价格',股票代码=i[0], 交易日期=i[1], 开盘价=i[2], 最高价=i[3], 最低价=i[4], 收盘价=i[5],成交量=i[6])
   graph.create(a)

In [9]:
matcher = NodeMatcher(graph)
#graph.run("MATCH (n)-[r]->() DELETE r")


In [10]:
# 企业-行业关系
industry_nodes = {}
for i in stock.values:
    stock_node = Node('股票', TS代码=i[0])
    industry_name = i[2]
    if industry_name in industry_nodes:
        industry_node = industry_nodes[industry_name]
    else:
        industry_node = Node('行业', 行业=industry_name)
        graph.create(industry_node)
        industry_nodes[industry_name] = industry_node
    graph.create(stock_node)
    r = Relationship(stock_node, '属于', industry_node)
    graph.create(r)


In [14]:
# 股东-股票关系
holder_nodes = {}
for i in holder.values:
    holder_name = i[3]
    stock_code = i[0]
    holding_quantity = i[4]
    holding_ratio = i[5]
    if holder_name not in holder_nodes:
        holder_node = Node('股东', 股东名称=holder_name)
        graph.create(holder_node)
        holder_nodes[holder_name] = holder_node
    stock_node = Node('股票', TS代码=stock_code)
    graph.create(stock_node)
    r = Relationship(holder_node, '持有股票x', stock_node, 持股数量=holding_quantity, 持股比例=holding_ratio)
    graph.create(r)
# 股东-股票关系
stock_nodes = {}  # 用于存储已创建的股票节点
for i in holder.values:
    holder_name = i[3]
    stock_code = i[0]
    if stock_code not in stock_nodes:
        stock_node = Node('股票', TS代码=stock_code)
        graph.create(stock_node)
        stock_nodes[stock_code] = stock_node

    holder_node = Node('股东', 股东名称=holder_name)
    graph.create(holder_node)
    r = Relationship(holder_node, '被持有x', stock_node, 持股数量=i[4], 持股比例=i[5])
    graph.create(r)


In [12]:
# 企业-概念
for i in concept.values:
   a = matcher.match("股票",TS代码=i[1]).first()  # 这里应该匹配股票代码，而不是概念名称
   b = matcher.match("概念",概念名称=i[0]).first()
   if a and b:
       r = Relationship(a,'概念属于',b)
       graph.create(r)


In [13]:
def calculate_return(df):
    df['return'] = (df['开盘价'] - df['收盘价']) / df['收盘价']
    return df

# 计算每只股票的收益率
price_grouped = price.groupby('股票代码').apply(calculate_return)

def compute_corrcoef(stock_return : np.ndarray):
    corrcoef = np.corrcoef(stock_return)
    corrcoef[np.isnan(corrcoef)] = 0 # nan的位置赋0
    return corrcoef

# 将数据转换成每一行包含所有股票在一天的收益率的格式
return_matrix = price_grouped.pivot(index='交易日期', columns='股票代码', values='return')

# 计算股票收益的皮尔逊相关系数
corr_matrix = compute_corrcoef(return_matrix.values)

# 定义相关系数阈值
threshold = 0.5

# 获取股票代码列表
stock_codes = return_matrix.columns.tolist()

# 遍历相关系数矩阵的上半部分
for i in range(corr_matrix.shape[0]):
    for j in range(i+1, corr_matrix.shape[1]):
        # 如果相关系数超过阈值
        if abs(corr_matrix[i, j]) > threshold:
            # 获取两个股票的节点
            a = matcher.match('股票', TS代码=stock_codes[i]).first()
            b = matcher.match('股票', TS代码=stock_codes[j]).first()
            if a and b:
                # 创建关系
                r = Relationship(a, '相关', b, corrcoef=corr_matrix[i, j])
                graph.create(r)

